# Nastya Chatbot Implementation

TODO write here

References:

- https://towardsdatascience.com/how-to-build-an-easy-quick-and-essentially-useless-chatbot-using-your-own-text-messages-f2cb8b84c11d

In [4]:
import pandas as pd
from string import punctuation

### 1. Getting the data to the format we want it to be in

Reading in the CSV file

In [5]:
df = pd.read_csv('data/message_data.csv', index_col=[0])
df.head()

,content,sender_name,timestamp_ms
0,Okay babes,Anastasiia Morozova,1582067120307
1,!!!,Pratik Karki,1582067116590
2,I just got done,Pratik Karki,1582067113534
3,I thought you were done!,Anastasiia Morozova,1582067104121
4,The person was a annoying ones,Pratik Karki,1582067101785


Sorting the dataframe by date

In [6]:
df.sort_values(by='timestamp_ms', inplace=True)
df.reset_index(inplace=True)
df.head()

,index,content,sender_name,timestamp_ms
0,182484,Cool,Anastasiia Morozova,1508118080364
1,182483,That was a quick reply,Pratik Karki,1508386653307
2,182482,Took me a while to realize how cool this fact is.,Anastasiia Morozova,1508417569122
3,182481,Lol,Pratik Karki,1508421997827
4,182480,That's kinda mean,Pratik Karki,1508422003040


Adding `is_from_me` column and dropping `index`

In [7]:
df['is_from_me'] = ['1' if x == 'Pratik Karki' else '0' for x in df['sender_name']]
df.drop('index', axis=1, inplace=True)
df.head()

,content,sender_name,timestamp_ms,is_from_me
0,Cool,Anastasiia Morozova,1508118080364,0
1,That was a quick reply,Pratik Karki,1508386653307,1
2,Took me a while to realize how cool this fact is.,Anastasiia Morozova,1508417569122,0
3,Lol,Pratik Karki,1508421997827,1
4,That's kinda mean,Pratik Karki,1508422003040,1


### 2. Now we can make training data

For this, we want to create a dataframe with `text` and `response` columns

In [8]:
#remove the nulls
df.dropna(subset = ['content'], inplace=True)

#helper functions
def make_sentences(series):
    return '. '.join(series)

#initiliaze empty df
train_data = pd.DataFrame(columns = ['text','response'])

In [16]:
test_df = pd.DataFrame(df).set_index('id')[:100].copy(deep=True)

Hello.. Its me


In [ ]:
# store current text and response sentences
text_sentence = []
response_sentence = []

# put all the weird characters here that we want to remove from the 
bad_strings = ["\u00e0", "\u00e4", "\u00e2", "\u00e7", "\u00e8", "\u00e9", "\u00ea","\u00eb", "\u00ee", "\u00ef"
               ,"\u00f4", "\u00f6", "\u00f9", "\u00fb", "\u00fc"]

#  	content	                         sender_name	     timestamp_ms	 is_from_me
#0 	Cool	                       Anastasiia Morozova	1508118080364	 0
#1 	That was a quick reply	         Pratik Karki	    1508386653307	 1
#

train_data_row = 0

#iterate through each convo
for index, row in df.iterrows():
    # retrieve current text and remove end punctuation
    curr_string = row['content'].rstrip(punctuation)
    
    if(row[is_from_me]):
        # going from response to text
        if len(response_sentence) > 0:
            sentence = make_sentence(response_sentence)
            train_data[train_data_row]['response'] = sentence
            response_sentence.clear()
            train_data_row+=1 # only increment when response is over
        text_sentence.append(curr_string)
    else:
        if len(text_sentence) > 0:
            sentence = make_sentence(text_sentence)
            train_data[train_data_row]['text'] = sentence
            text_sentence.clear()
        response_sentence.append(curr_string)

        